# Preparation

In [14]:
# IMPORT NECESSARY LIBS

import numpy as np
import os
import tqdm
import copy
from PIL import Image, ImageFilter
import matplotlib.cm as mpl_color_map

import torch
from torch.autograd import Variable
from torchvision import models
from torch.optim import Adam
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
# MOUNT GOOGLE DRIVE

from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


# Build the model

In [3]:
# VGG11 MODEL

# VGG11
class VGG(nn.Module):
    def __init__(self, features, output_dim):
        super().__init__()
        
        self.features = features
        
        self.avgpool = nn.AdaptiveAvgPool2d(7)
        
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(inplace = True),
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace = True),
            nn.Dropout(0.5),
            nn.Linear(4096, output_dim),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        h = x.view(x.shape[0], -1)
        x = self.classifier(h)
        return x, h

# Config for VGG11
vgg11_config = [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M']

# Build the model from config file
def get_vgg_layers(config, batch_norm):
    
    layers = []
    in_channels = 3
    
    for c in config:
        assert c == 'M' or isinstance(c, int)
        if c == 'M':
            layers += [nn.MaxPool2d(kernel_size = 2)]
        else:
            conv2d = nn.Conv2d(in_channels, c, kernel_size = 3, padding = 1)
            if batch_norm:
                layers += [conv2d, nn.BatchNorm2d(c), nn.ReLU(inplace = True)]
            else:
                layers += [conv2d, nn.ReLU(inplace = True)]
            in_channels = c
            
    return nn.Sequential(*layers)

vgg11_layers = get_vgg_layers(vgg11_config, batch_norm = True)

# Basic methods

In [5]:
# SAVE IMAGE

def save_image(im, path):
  
    if isinstance(im, (np.ndarray, np.generic)):
        im = format_np_output(im)
        im = Image.fromarray(im)
    im.save(path)

In [6]:
# PREPROCESS IMAGE

def preprocess_image(pil_im, resize_im=True):

    if type(pil_im) != Image.Image:
        try:
            pil_im = Image.fromarray(pil_im)
        except Exception as e:
            print("could not transform PIL_img to a PIL Image object. Please check input.")

    # Resize image
    if resize_im:
        pil_im = pil_im.resize((224, 224), Image.ANTIALIAS)

    # Convert image to D,W,H array
    im_as_arr = np.float32(pil_im)
    im_as_arr = im_as_arr.transpose(2, 0, 1)
    
    # Normalize the channels
    for channel, _ in enumerate(im_as_arr):
        im_as_arr[channel] /= 255

    im_as_ten = torch.from_numpy(im_as_arr).float()
    im_as_ten.unsqueeze_(0)
    im_as_var = Variable(im_as_ten, requires_grad=True)
    
    return im_as_var


In [7]:
# CONVERT TO 3xWxH format

def format_np_output(np_arr):
    
    # Phase/Case 1: The np arr only has 2 dimensions
    if len(np_arr.shape) == 2:
        np_arr = np.expand_dims(np_arr, axis=0)

    # Phase/Case 2: Np arr has only 1 channel (assuming first dim is channel)
    if np_arr.shape[0] == 1:
        np_arr = np.repeat(np_arr, 3, axis=0)

    # Phase/Case 3: Np arr is of shape 3xWxH
    if np_arr.shape[0] == 3:
        np_arr = np_arr.transpose(1, 2, 0)

    # Phase/Case 4: NP arr is normalized between 0-1
    if np.max(np_arr) <= 1:
        np_arr = (np_arr*255).astype(np.uint8)
        
    return np_arr

In [8]:
# GET EXAMPLES, SET PATHES, LOAD BEST WEIGHTS

def get_example_params(example_index):

    # 0 - covid; 1 - normal
    root_path = '/content/drive/MyDrive/Egyetem/Szakdolgozat/Code/Datasets/'
    root_path = root_path + 'CXR/' if cxr is True else root_path + 'CT/'
    root_path = root_path + 'test/'
    covid_path = root_path + 'covid-19/'
    normal_path = root_path + 'normal/'

    # CXR database
    example_list_cxr = (('/content/drive/MyDrive/Egyetem/Szakdolgozat/Code/Datasets/CXR/test/normal/normal_913.png', 1),
                        ('/content/drive/MyDrive/Egyetem/Szakdolgozat/Code/Datasets/CXR/test/covid/covid_889.png', 0),
                        ('/content/drive/MyDrive/Egyetem/Szakdolgozat/Code/Datasets/CXR/test/covid/covid_906.png', 0),
                        ('/content/drive/MyDrive/Egyetem/Szakdolgozat/Code/Datasets/CXR/test/normal/normal_894.png', 1),
                        ('/content/drive/MyDrive/Egyetem/Szakdolgozat/Code/Datasets/CXR/test/normal/normal_966.png', 1))
    
    # CT database
    example_list_ct = (('/content/drive/MyDrive/Egyetem/Szakdolgozat/Code/Datasets/CT/test/covid/covid_322.png', 0),
                       ('/content/drive/MyDrive/Egyetem/Szakdolgozat/Code/Datasets/CT/test/covid/covid_328.png', 0),
                    ('/content/drive/MyDrive/Egyetem/Szakdolgozat/Code/Datasets/CT/test/covid/covid_344.png', 0),
                    ('/content/drive/MyDrive/Egyetem/Szakdolgozat/Code/Datasets/CT/test/normal/normal_361.png', 1),
                    ('/content/drive/MyDrive/Egyetem/Szakdolgozat/Code/Datasets/CT/test/normal/normal_386.png', 1))
    
    example_list = example_list_cxr if cxr is True else example_list_ct
    img_path = example_list[example_index][0]
    target_class = example_list[example_index][1]
    file_name_to_export = img_path[img_path.rfind('/')+1:img_path.rfind('.')]
    
    # Read image
    original_image = Image.open(img_path).convert('RGB')
    original_image = original_image.resize((224, 224), Image.ANTIALIAS)

    
    # Process image
    prep_img = preprocess_image(original_image)
    
    # Define model
    OUTPUT_DIM = 2
    pretrained_model = VGG(vgg11_layers, OUTPUT_DIM)
    path_cxr = "/content/drive/MyDrive/Egyetem/Szakdolgozat/Code/Classification/checkpoints/best_cxr_classifier_0_99.pt"
    path_ct = '/content/drive/MyDrive/Egyetem/Szakdolgozat/Code/Classification/cp_teszt/best_ct_classifier_5_81.pt'
    checkpoint = torch.load(path_ct, map_location ='cpu')
    pretrained_model.load_state_dict(checkpoint)

    return (original_image,
            prep_img,
            target_class,
            file_name_to_export,
            pretrained_model)

In [9]:
# SAVE IMAGE AND ACTIVATION HEATMAP ON IMAGE

def save_class_activation_images(org_img, activation_map, file_name, tl, first=False):

    # Set pathes
    result_path = '/content/drive/MyDrive/Egyetem/Szakdolgozat/Code/GradCam/results/'
    if not os.path.exists(result_path):
        os.makedirs(result_path)
    result_path = result_path + 'CXR/' if cxr is True else result_path + 'CT/'
    result_path = result_path + 'covid' if target_class is 0 else result_path + 'normal'

    heatmap, heatmap_on_image = apply_colormap_on_image(org_img, activation_map, 'hsv')
    
    # Save original image
    path_to_file = os.path.join(result_path, file_name+'.png')
    if first:
        save_image(org_img, path_to_file)

    # Save heatmap on iamge
    path_to_file = os.path.join(result_path, file_name+tl+'_Cam_On_Image.png')
    save_image(heatmap_on_image, path_to_file)

In [10]:
# APPLY HEATMAP ON IMAGE

def apply_colormap_on_image(org_im, activation, colormap_name):
    
    # Get colormap
    color_map = mpl_color_map.get_cmap(colormap_name)
    no_trans_heatmap = color_map(activation)

    heatmap = copy.copy(no_trans_heatmap)
    heatmap[:, :, 3] = 0.4
    heatmap = Image.fromarray((heatmap*255).astype(np.uint8))
    no_trans_heatmap = Image.fromarray((no_trans_heatmap*255).astype(np.uint8))

    heatmap_on_image = org_im
    heatmap_on_image = heatmap_on_image.convert('RGBA')
    heatmap_on_image = Image.alpha_composite(heatmap_on_image, heatmap)
    return no_trans_heatmap, heatmap_on_image

# Produce CAM (Class Activation Map)

In [11]:
# EXTRACT CAM FEATURES FROM THE MODEL

class CamExtractor():
    def __init__(self, model, target_layer):
        self.model = model
        self.target_layer = target_layer
        self.gradients = None

    def save_gradient(self, grad):
        self.gradients = grad

    # Save the convolution output on that layer
    def forward_pass_on_convolutions(self, x):
        conv_output = None
        for module_pos, module in self.model.features._modules.items():
            x = module(x)
            if int(module_pos) == self.target_layer:
                x.register_hook(self.save_gradient)
                conv_output = x
        return conv_output, x

    # Forward pass on the convolutions
    def forward_pass(self, x):
        conv_output, x = self.forward_pass_on_convolutions(x)
        x = x.view(x.size(0), -1)
        x = self.model.classifier(x)
        return conv_output, x

In [17]:
# GENERATE CLASS ACTIVATION MAP

class GradCam():

    def __init__(self, model, target_layer):
        self.model = model
        self.model.eval()
        self.extractor = CamExtractor(self.model, target_layer)

    def generate_cam(self, input_image, target_class=None):

        # conv_output is the output of convolutions at specified layer
        # model_output is the final output of the model
        conv_output, model_output = self.extractor.forward_pass(input_image)
        if target_class is None:
            target_class = np.argmax(model_output.data.numpy())
        
        # Target for backprop
        one_hot_output = torch.FloatTensor(1, model_output.size()[-1]).zero_()
        one_hot_output[0][target_class] = 1
        # Zero grads
        self.model.features.zero_grad()
        self.model.classifier.zero_grad()
        
        # Backward pass with specified target
        model_output.backward(gradient=one_hot_output, retain_graph=True)
        # Get hooked gradients
        guided_gradients = self.extractor.gradients.data.numpy()[0]
        # Get convolution outputs
        target = conv_output.data.numpy()[0]
        # Get weights from gradients
        weights = np.mean(guided_gradients, axis=(1, 2))
        cam = np.ones(target.shape[1:], dtype=np.float32)

        # Multiply each weight with its conv output and then sum
        for i, w in enumerate(weights):
            cam += w * target[i, :, :]
        cam = np.maximum(cam, 0)
        # Normalize between 0-1
        cam = (cam - np.min(cam)) / (np.max(cam) - np.min(cam))
        # Scale between 0-255 to visualize
        cam = np.uint8(cam * 255)
        cam = np.uint8(Image.fromarray(cam).resize((input_image.shape[2],
                       input_image.shape[3]), Image.ANTIALIAS))/255

        return cam

# Execute

In [18]:
    # Get params
    cxr = False
    target_example = 0
    (original_image, prep_img, target_class, file_name_to_export, pretrained_model) =\
        get_example_params(target_example)
    
    # Conv2d layers: 0, 4, 8, 11, 15, 18, 22, 25
    conv_layers = [4, 8, 11, 15, 18, 22, 25]
    for target_layer in conv_layers:
        
        # Set local params
        isFirst = True if target_layer == conv_layers[0] else False
        layer_id = "_" + str(target_layer)
        
        # Init grad cam and generate mask
        grad_cam = GradCam(pretrained_model, target_layer=target_layer)
        cam = grad_cam.generate_cam(prep_img, target_class)

        # Save mask
        save_class_activation_images(original_image, cam, file_name_to_export, layer_id, isFirst)